# Capturando tuits

[Pablo Haya](http://pablohaya.com), Ultima versión: Mar-2018

## Objetivo

Este _notebook_ presenta como utiliza Python para obtener tuits a través de la API pública de Twitter. Es preciso tener instalada ...

### Obtención de credenciales de acceso


La captura de tuits se va realizar empleando la API pública que provee Twitter. Antes de empezar es preciso dar alta una aplicación en Twitter, y obtener las credenciasles de acceso que autoricen a la aplicación a descargarse menciones desde la API.

Los pasos para dar de alta una aplicación son:

* Crea una cuenta en [Twitter](https://twitter.com/) si no se dispusiera antes.
* Ve a https://apps.twitter.com/ y conectate con tu usuario.
* Haz click en _Create New App_.
* Rellena el formulario, incluido la dirección de website, aunque no se disponga.
* Acepta los términos y condiciones y crearla.
* En la página siguiente, haz click en _Keys and Access Tokens_ y copia tu _API key_ y _API secret_.
* Baja al final de la página, haz clic en _Create my access token_, y copia _Access token_ y _Access token secret_ en los huecos correspondientes.

### Configuración de la conexión a la API

Una vez creada la aplicación pasamos al script propiamente dicho que primeramente requiere importar la bibliotecas necesarias. Para poder acceder a la API de Twitter se va a utilizar la biblioteca [tweepy](http://www.tweepy.org/). También son necesarias la biblioteca `json`, para manipular los tweets en este formato, y la biblioteca `time` como veremos más adelante.

In [1]:
# Bibliotecas necesarias
import json
import time
import tweepy
from tweepy.streaming import StreamListener

Es preciso añadir las credenciales obtenidas al crear la aplicación sustituyéndolas en el siguiente extracto de código: 

In [2]:
# Credenciales de acceso a la API de Twitter
access_token = "INSERT ACCESS TOKEN"
access_token_secret = "INSERT ACCESS TOKEN SECRET"
consumer_key = "INSERT CONSUMER KEY"
consumer_secret = "INSERT CONSUME SECRET"

Se realiza la autenticación empleando las credeciales definidas anteriormente, y se inicializa la conexión.

In [3]:
# Autenticación
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

### Extraer tuits de la propia cronología

La opción más directa para obtener un conjunto de tuits es acceder a la cronología (_timeline_) del propio usuario que ha creado la aplicación. Por defecto, la función [home_timeline](http://docs.tweepy.org/en/v3.5.0/api.html#API.home_timeline) permite acceder a los últimos 20 tuits publicados en la cronología.

La descarga no es inmediata de manera que es posible que tarde unos segundos en aparecer el resultado. Mientras que se estén capturando aparecerá un asterisco [*] en vez del identificador de la celda. 

In [4]:
api = tweepy.API(auth)

# Función que transforma un objeto Status en json
# No está documentado en la API
def to_json(status):    
    json_str = json.dumps(status._json)
    return json_str

# Función que escribe en un fichero un conjuto de tuits en formato json
# Los tuits se añaden al fichero, y en caso de que no existe se crea
def write_json(tweets, filename):
    n = 0
    try:
        with open(filename, 'a') as outfile:
            for tweet in tweets:
                outfile.write(to_json(tweet))
                n = n + 1
    except (IOError, OSError, Failure) as e:
        pass
    finally:
        return n

In [5]:
public_tweets = api.home_timeline()
n = write_json(public_tweets, 'home_timeline.json')
print("Guardados %d tuits" % n)

# Imprime únicamente el texto cada tuit a modo de comprobación
for tweet in public_tweets:
    print(tweet.text)

Guardados 20 tuits
8 técnicas para superar el miedo a hablar en público #Tips https://t.co/MQ92U8TaPU
Qué es el pulido corporal y por qué es importante https://t.co/1uF7KIQk3Y
¿Estamos al borde de una EXTINCIÓN masiva? [VÍDEO] #ciencia https://t.co/hv5jTBkMkU
Así es Bossòst, el pueblo más antiindependentista de Cataluña [VIDEO] https://t.co/SiBHokSDs4
RT @LogiNext: $1.2 trillion to run through insights driven business by 2020: @forrester #BusinessIntelligence #GrowthHacking #DataAnalytics…
La clave para bajar de peso es la calidad de los alimentos, no la cantidad #Nutrición https://t.co/qFiJnLpy8w
Estos son los 15 idiomas que más se hablan en el mundo #cultura https://t.co/uFTnnAd9nd
El descubrimiento confirmado de la ‘segunda tierra’ ¿Por qué su gran importancia? #planeta https://t.co/I1zFPpKjY5
Advierten una ‘peligrosa amenaza’ de los teléfonos móviles (aunque estén apagados) #Investigación https://t.co/NG4RpFQXxs
Pirulito testimonial pasquale! (Pirulito testimonial pasquale!) https:

### Acceder a los tuits publicados por un  usuario

Es igual de fácil acceder a los tuits publicados por usuario mediante la función [user_timeline](http://docs.tweepy.org/en/v3.5.0/api.html#API.user_timeline). En este caso nos vamos a descargar los tuits [KDnuggets (@kdnuggets)](https://twitter.com/kdnuggets). El nombre de usuario se añade como parámetro de la función sin la @. El [máximo número de tuits](# https://developer.twitter.com/en/docs/tweets/timelines/api-reference/get-statuses-user_timeline.html) que se pueden descargar en cada llamada es de 3.200

In [67]:
# Nos descargamos los 150 últimos tuits del timeline
public_tweets = api.user_timeline("KDnuggets", count=150)
n = write_json(public_tweets, 'KDnuggets_timeline.json')
print("Guardados %d tuits" % n)

# Imprimos únicamente los 20 primeros tuits
for tweet in public_tweets[1:20]:
    print(tweet.text)


Guardados 150 tuits
Learn how make great #visualizations using Dash with advanced #DataVisualization workshops for Dash, R #rstats, Shi… https://t.co/JfPxK6IJ4R
Introduction to Functional Programming in #Python #KDN https://t.co/GAFiBCnHe8
World Models- Can agents learn inside of their own dreams? https://t.co/t8a2r9JqYz https://t.co/CIHEo0gxpY
Computer Vision by Andrew Ng - 11 Lessons Learned #KDN https://t.co/HYOG3xufYk
5 Things to Know About #MachineLearning https://t.co/JnLBN9rsNp https://t.co/WF0gYwII63
What worries me about #AI – François Chollet – Medium https://t.co/Ry3n421cBt https://t.co/gJ0bblpQNq
Quantum #MachineLearning: An Overview #KDN https://t.co/ZpIkBYbYoF
Using Machine Learning to Discover #NeuralNetwork Optimizers https://t.co/oo1LhhM3gm via @googleresearch blog https://t.co/oXvUlpaL7a
The 10 Statistical Techniques #DataScientists Need to Master #KDN https://t.co/yxzS9Uu9qf
Didactical Employee Attrition Kernel https://t.co/Y5c8e1lFHF https://t.co/MmoOHv5YbI
Top 12 E

### Obteniendo cualquier tuit 

En general, podemos preguntar a la API para poder descargarnos cualquier tuit que contenga los términos queramos medianta la función [search](http://docs.tweepy.org/en/v3.5.0/api.html#API.search). Es importante tener en cuenta que la API pública tiene ciertas restricciones que impide descargarse todos los posibles tuits cuando el volumen es muy elevado.

Para ello se define un consulta que se codifica como un array de términos que se interpreta como un OR. Así para descargar tuits que contenga el término "hadoop" o "spark", se codificaría como `["hadoop", "spark"]` 

También se pueden incluir términos relacionados mediante un AND si se separan con un espacio, de manera que es preciso que estén ambos términos para que se capture el tuit. En este caso, sería necesario especificar mejor que tuits habría capturar relativos a _Spark_, ya que es un palabra que puede aparecer en otros contextos distintos de tecnologías de Big Data.

Por ejemplo: si quisiéramos la siguiente consulta "hadoop" OR ("apache" AND "spark") se codificará como `["hadoop", "apache spark"]`.

In [11]:
public_tweets = api.search(["hadoop", "apache spark"])
n = write_json(public_tweets, 'hadoop_or_apachespark.json')
print("Guardados %d tuits" % n)

for tweet in public_tweets:
    print(tweet.text)

Guardados 10 tuits
RT @techjunkiejh: Predicting Breast Cancer Using Apache Spark Machine Learning Logistic Regression | MapR https://t.co/FSwMwGTmzU #hadoop #…
RT @techjunkiejh: Predicting Breast Cancer Using Apache Spark Machine Learning Logistic Regression | MapR https://t.co/FSwMwGTmzU #hadoop #…
RT @techjunkiejh: Predicting Breast Cancer Using Apache Spark Machine Learning Logistic Regression | MapR https://t.co/FSwMwGTmzU #hadoop #…
Predicting Breast Cancer Using Apache Spark Machine Learning Logistic Regression | MapR https://t.co/FSwMwGTmzU… https://t.co/cnWh0oYw7N
RT @tmccuch: What is Apache Hadoop HDFS Ozone? https://t.co/rbttC6YLvk &lt;-- Ozone is a redundant, distributed Object Store for Hadoop. Hive,…
RT @tmccuch: What is Apache Hadoop HDFS Ozone? https://t.co/rbttC6YLvk &lt;-- Ozone is a redundant, distributed Object Store for Hadoop. Hive,…
RT @tmccuch: What is Apache Hadoop HDFS Ozone? https://t.co/rbttC6YLvk &lt;-- Ozone is a redundant, distributed Object Store for Hado

### Capturando tuits en tiempo real

A veces queremos capturar tuit de manera continuada. Para ello podemos definir una consulta que se descarge un flujo de tuits a medida que se van publicando en Twitter.

Primeramente hay que declararse una clase de tipo `StreamListener` que gestione el flujo de tuits defiendo que se quiere hacer con cada uno, y cuando se termina la captura. 

La siguiente clase se encarga de guardar en fichero cada tuit recibido. Se inicializa con el nombre del fichero, y con el tiempo durante el cual se estarán capturando los tuits. Cada vez que se recibe un tuit se llama el método `on_data` que se encarga de comprobar si se ha excedido la duración de la captura, y de guardar el tuit en el fichero. El retorno de este función indica si se continua `True` o no `False` con la descarga de tweets.

In [8]:
class FileStreamListener(StreamListener):
    # requiere el nombre del fichero donde almacenar los tweets, 
    # y el tiempo límite en segundos con el que se configura la captura.
    def __init__(self, filename, time_limit=30):
        self.start_time = time.time()
        self.limit = time_limit
        self.save_file = open(filename, 'a')
        self.n = 0 # numero de tuits procesados
        super(FileStreamListener, self).__init__()

    def on_data(self, tweet):
        if (time.time() - self.start_time) < self.limit:
            #print(tweet)
            self.save_file.write(tweet)
            self.save_file.write('\n')
            self.n = self.n + 1
            return True
        else:
            self.save_file.close()
            return False

A continuación se inicializa el flujo de tuits, y se configura con una consulta similar a la hicimos anteriormente para extraer tuits sobre "Hadoop" y "Apache Spark".

Recordar que es preciso esperar hasta que venza el tiempo de la captura definido (1 min.) para disponer del archivo con los tweets. Mientras que se estén capturando aparecerá un asterisco [*] en vez del identificador de la celda.  

In [9]:
# Conexión a stream data
listener = FileStreamListener(filename='twitter_stream_data.json', time_limit=60*15)
stream = tweepy.Stream(auth, listener)
# Captura del stream de Twitter actualizaciones de estado que contenga los términos
stream.filter(track = ['hadoop', 'apache spark'])

In [10]:
print("Guardados %d tuits" % listener.n)

Guardados 2 tuits


### Capturando tendencias en Twitter

Para terminar vamos a ver como se puede obtener aquellos términos que están siendo tendencia (_trending topics_) en un momento dado. Hay que tener en cuenta que las tendencias se definen por zonas geográficas de manera que es precio especificar primeramente la región de interés sobre la cual se quieren obtener las tendencias. Cada región se identifica con un _woeid_. Por ejemplo, los _trending topics_ mundiales son el _woeid_ 1.

A continuación se consultan todas las posibles regiones que existen:

In [61]:
locations = api.trends_available()
locations

Guardados 10 tuits


[{'country': '',
  'countryCode': None,
  'name': 'Worldwide',
  'parentid': 0,
  'placeType': {'code': 19, 'name': 'Supername'},
  'url': 'http://where.yahooapis.com/v1/place/1',
  'woeid': 1},
 {'country': 'Canada',
  'countryCode': 'CA',
  'name': 'Winnipeg',
  'parentid': 23424775,
  'placeType': {'code': 7, 'name': 'Town'},
  'url': 'http://where.yahooapis.com/v1/place/2972',
  'woeid': 2972},
 {'country': 'Canada',
  'countryCode': 'CA',
  'name': 'Ottawa',
  'parentid': 23424775,
  'placeType': {'code': 7, 'name': 'Town'},
  'url': 'http://where.yahooapis.com/v1/place/3369',
  'woeid': 3369},
 {'country': 'Canada',
  'countryCode': 'CA',
  'name': 'Quebec',
  'parentid': 23424775,
  'placeType': {'code': 7, 'name': 'Town'},
  'url': 'http://where.yahooapis.com/v1/place/3444',
  'woeid': 3444},
 {'country': 'Canada',
  'countryCode': 'CA',
  'name': 'Montreal',
  'parentid': 23424775,
  'placeType': {'code': 7, 'name': 'Town'},
  'url': 'http://where.yahooapis.com/v1/place/3534',

Nos fijamos en una localización en concreto (por ejemplo, Madrid)

In [62]:
for loc in locations:
    if loc['name'] == 'Madrid':
        mad_woeid = loc['woeid']
        break
mad_woeid

766273

Obtenemos todas las tendencias de Madrid

In [63]:
mad_tt = api.trends_place(mad_woeid)
mad_tt

[{'as_of': '2018-03-29T21:41:51Z',
  'created_at': '2018-03-29T21:36:09Z',
  'locations': [{'name': 'Madrid', 'woeid': 766273}],
  'trends': [{'name': 'Semana Santa',
    'promoted_content': None,
    'query': '%22Semana+Santa%22',
    'tweet_volume': 294127,
    'url': 'http://twitter.com/search?q=%22Semana+Santa%22'},
   {'name': '#SVGala3',
    'promoted_content': None,
    'query': '%23SVGala3',
    'tweet_volume': 28527,
    'url': 'http://twitter.com/search?q=%23SVGala3'},
   {'name': '#Gladiator',
    'promoted_content': None,
    'query': '%23Gladiator',
    'tweet_volume': None,
    'url': 'http://twitter.com/search?q=%23Gladiator'},
   {'name': '#SelfiesforMimi',
    'promoted_content': None,
    'query': '%23SelfiesforMimi',
    'tweet_volume': None,
    'url': 'http://twitter.com/search?q=%23SelfiesforMimi'},
   {'name': '#FirstDates598',
    'promoted_content': None,
    'query': '%23FirstDates598',
    'tweet_volume': None,
    'url': 'http://twitter.com/search?q=%23First

Recorremos las tendencias, y nos descargamos los últimos tuits para cada una de ellas a partir del atributo ´query´, y los guardamos en fichero.

In [64]:
# obtenemos todas las queries
queries_tt = list(map(lambda tweet: tweet['query'], mad_tt[0]['trends']))

for q in queries_tt:
    public_tweets = api.search(q)
    write_json(public_tweets, 'trending_topics.json')

# Imprimimos unicamente los tuits de la última query
for tweet in public_tweets:
    print(tweet.text)

@uscl_football @RMCsport L'époque où on espérait la #Ligue1 🤩
RT @infosportplus: Depuis son retour parmi l'élite en 2013, @AS_Monaco figure parmi les meilleures formations en ayant toujours fini sur le…
RT @sportnut89: Which of these are you most looking forward to this weekend? 1/2 #BSB #BritGT #F3 #LaLiga #SerieA #Ligue1 #Bundesliga #SPL…
RT @sportnut89: Which of these are you most looking forward to this weekend? 2/2 #BSB #BritGT #F3 #LaLiga #SerieA #Ligue1 #Bundesliga #SPL…
RT @sportnut89: Which of these are you most looking forward to this weekend? 1/2 #BSB #BritGT #F3 #LaLiga #SerieA #Ligue1 #Bundesliga #SPL…
RT @infosportplus: Souvent remplaçant mais dernièrement décisif avec @OM_Officiel, l'attaquant K. #Mitroglou 🇬🇷 affiche pourtant un beau ra…
RT @infosportplus: Révélation du club la saison passée, le jeune milieu de terrain de @OM_Officiel Maxime #Lopez brille moins cette année.…
RT @infosportplus: Du "Big 4", @OM_Officiel est l'équipe qui a déjà joué le plus de matches et q

### Ejercicios

1. Modificar la descarga de tuits en tiempo real de manera incluyan los términos "logistic regression" o "deep learning".
* Crear un nuevo `StreamListener` que termine de capturar tuits cuando haya descargado un número predeterminado. 
* Recuperar todos los _trending topics_ de España, Madrid y Barcelona en un único archivo.
